In [1]:
from fastkaggle import *
from fastai import *
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

/home/carson/mambaforge/envs/fastai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#fastkaggle setup
comp = "store-sales-time-series-forecasting"

path = setup_comp(comp)
path

Path('store-sales-time-series-forecasting')

# Data Setup

In [12]:
holiday_events_df = pd.read_csv(path/'holidays_events.csv')
oil_df = pd.read_csv(path/'oil.csv')
stores_df = pd.read_csv(path/'stores.csv')
test_df = pd.read_csv(path/'test.csv')
train_df = pd.read_csv(path/'train.csv')
transactions_df = pd.read_csv(path/'transactions.csv')


#correct datetimes
# train_df["date"] = pd.to_datetime(train_df.date)
# test_df["date"] = pd.to_datetime(test_df.date)
# transactions_df["date"] = pd.to_datetime(transactions_df.date)
add_datepart(train_df, 'date')
add_datepart(test_df, 'date')
drop_cols = ['id', 'Elapsed']
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)

#categorize familys
train_df['family'] = train_df['family'].astype("category")
train_df['family'] = train_df.family.cat.codes


#correct datatypes
# train_df.onpromotion = train_df.onpromotion.astype("float16")
# train_df.sales = train_df.sales.astype("float32")
# stores_df.cluster = stores_df.cluster.astype("int8")

In [13]:
print(train_df.dtypes)

store_nbr             int64
family                 int8
sales               float64
onpromotion           int64
Year                  int64
Month                 int64
Week                  int64
Day                   int64
Dayofweek             int64
Dayofyear             int64
Is_month_end           bool
Is_month_start         bool
Is_quarter_end         bool
Is_quarter_start       bool
Is_year_end            bool
Is_year_start          bool
dtype: object


In [17]:
from fastai.tabular.all import *
cats = ['family', 'Year', 'Month', 'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'onpromotion']
conts = ['store_nbr']
# conts, cats = cont_cat_split(train_df, dep_var='sales')
procs = [Categorify, Normalize, FillMissing]
splits = RandomSplitter()(range_of(train_df))
dls = TabularDataLoaders.from_df(train_df, procs=procs, cat_names=cats, cont_names=conts, 
                                  y_names="sales", bs=128)
learn = tabular_learner(dls, metrics=accuracy) 

In [16]:
print(f'Conts: {conts}\n Cats: {cats}')

Conts: ['store_nbr', 'family', 'onpromotion', 'Week', 'Day', 'Dayofyear']
 Cats: ['Year', 'Month', 'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']


In [18]:
dls.show_batch()
learn.fit_one_cycle(1) #todo figure out how to get this a litttttllleee nicer

,family,Year,Month,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,onpromotion,store_nbr,sales
0,4,2017,3,0,False,False,False,False,False,False,0,2.000001,1.0
1,25,2015,11,5,False,False,False,False,False,False,3,45.000000,804.0
2,16,2014,4,1,False,False,False,False,False,False,0,45.000000,0.0
3,16,2016,8,5,False,False,False,False,False,False,0,14.000000,12.0
4,16,2014,2,1,False,False,False,False,False,False,0,31.000000,0.0
5,32,2016,9,5,False,False,False,False,False,False,0,25.000000,0.0
6,23,2014,8,0,False,False,False,False,False,False,0,51.000000,0.0
7,18,2016,4,4,False,False,False,False,False,False,0,37.000000,231.0
8,1,2016,2,6,False,False,False,False,False,False,0,44.000000,0.0
9,1,2017,3,2,False,False,False,False,False,False,0,32.000000,0.0


epoch,train_loss,valid_loss,accuracy,time
0,364643.312500,259303.140625,0.312559,01:53


In [19]:
learn.show_results()

,family,Year,Month,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,onpromotion,store_nbr,sales,sales_pred
0,9.0,3.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.545091,290.000000,277.590027
1,18.0,2.0,5.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.545091,0.000000,-18.324602
2,8.0,2.0,5.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.251007,3298.000000,1623.441895
3,19.0,1.0,8.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.865822,0.000000,-28.065279
4,5.0,2.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.635885,0.000000,-38.985641
5,31.0,3.0,12.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.160213,912.382019,799.645874
6,18.0,5.0,8.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.224359,0.000000,23.242926
7,4.0,4.0,4.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.571739,0.000000,1166.938965
8,24.0,4.0,4.0,5.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,-1.378993,4.000000,6.561602


In [23]:
train_df['sales'].describe()

count    3.000888e+06
mean     3.577757e+02
std      1.101998e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.100000e+01
75%      1.958473e+02
max      1.247170e+05
Name: sales, dtype: float64

In [33]:
print(train_df['sales'][train_df['sales'].idxmax(axis=0)], train_df['sales'].max())

124717.0 124717.0
